---
title: Retrieval of effective Leaf and Soil temperatures with UAV imagery
subject: Tutorial
subtitle: Notebook that shows one method to prepare soil and canopy teperature maps with very high spatial resolution imagery
short_title: UAV component Tc and Ts
authors:
  - name: Héctor Nieto
    affiliations:
      - Instituto de Ciencias Agrarias, ICA
      - CSIC
    orcid: 0000-0003-4250-6424
    email: hector.nieto@ica.csic.es
  - name: Benjamin Mary
    affiliations:
      - Insituto de Ciencias Agrarias
      - CSIC
    orcid: 0000-0001-7199-2885
license: CC-BY-SA-4.0
keywords: UAV, TSEB, LST-NDVI space
---

# Summary
This interactive Jupyter Notebook has the objective of showing one method to extract effective leaf and soil temperatures from UAV imagery, which could be use then to run [TSEB-2T](https://github.com/hectornieto/pyTSEB/blob/382e4fc01e965143ebafdaefe5be9b45c737455a/pyTSEB/TSEB.py#L144). It is based on [](https://doi.org/10.1007/s00271-018-0585-9).

# Instructions
Read carefully all the text and follow the instructions.

Once each section is read, run the jupyter code cell underneath (marked as `In []`) by clicking the icon `Run`, or pressing the keys SHIFT+ENTER of your keyboard. A graphical interface will then display, which allows you to interact with and perform the assigned tasks.

To start, please run the following cell to import all the packages required for this notebook. Once you run the cell below, an acknowledgement message, stating all libraries were correctly imported, should be printed on screen.

In [ ]:
%matplotlib inline
%matplotlib widget
from pathlib import Path
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display
import numpy as np
print('libraries imported correctly')

# The leaf and soil temperatures in TSEB
TSEB the evapotranspiration (ET) and energy fluxes from vegetation and soil components providing the capability for estimating soil evaporation (E) and canopy transpiration (T). However, it is crucial for ET partitioning to retrieve reliable estimates of leaf/canopy ($T_C$) and soil ($T_S$) temperatures, in order to retrieve soil and canopy sensible heat fluxes:

:::{math}
:label:ec-hc-hs
H_C &= \rho C_p \frac{T_C - T_{ac}} {R_x}\\
H_S &= \rho C_p \frac{T_S - T_{ac}} {R_s}
:::

The use of very high spatial TIR imagery can be especially relevant in row crops with wide spacing and strongly clumped vegetation such as vineyards and orchards, since it is possible to directly separate these component temperatures and thus avoid the use of the assumptions made in TSEB-PT (or in TSEB-SW) regarding the initialization of potential transpiration.

:::{important}
Despite UAV technology can provide very high spatial resolution information we have to be aware that TSEB, as most ot the resistance-based energy balance model, must be applied at an adequate spatial domain to comply to the assumptions and the physical formulations inherent in it. Therefore **it is not possible** to run TSEB at leaf-level scale, as the formulations of radiative transfer and turbulent exchanche that it relies upon, require to run TSEB at a scale in the order of meters.
:::

# Separation of $T_C$ and $T_S$ 
In this exercise, we will retrieve the effective leaf and soil temperatures by searching for pure vegetation and soil pixels in a contextual spatial domain.

## Dataset
The UAV data correspons to the same acquisition as the [Running pyTSEB with an image](./.301-pyTSEB_image_configuration_file.ipynb] notebook.

These data were collected in 2014 during the early August IOP by the Aggie Air Team. The time of acquisition was approximately during the Landsat 7/8 overpass time. The UAV system flew at 450 m agl, resulting in 0.15 m pixel resolution in the visible and near-infrared bands and 0.60 m resolution in the thermal infrared. The visible and near-infrared sensors wavebands are similar to the Landsat blue, green, red, and near-infrared channels.

:::{seealso}
:class:dropdown
For more details see the [GRAPEX special issue in Irrigation Science published in 2019](https://doi.org/10.1007/s00271-019-00633-7) and [](https://doi.org/10.3390/s17071499).
:::

We will use Python GDAL to open both the LST and the NDVI images. Note that the NDVI was resampled to 0.6 in order to match the LST spatial resolution.

:::{seealso}
You can also open these images in QGIS:
* the LST raster is at [](./input/pyTSEB/SLM_001_002_20140809_1041_TIR.tif)
* the NDVI raster at the 0.15m resolution is at [](./input/pyTSEB/SLM_001_002_20140809_1041_NDVI.tif)
* the ressampledNDVI raster at the LST resolution is at [](./input/pyTSEB/SLM_001_002_20140809_1041_NDVI-LR.tif)
:::

In [ ]:
from pathlib import Path
import numpy as np
from osgeo import gdal

workdir = Path()
inputdir = workdir / "input" / "UAV"

#inputdir = workdir / "dataset" / "uav_imagery"

# Set the path to the LST and NDVI images
lst_file = inputdir / "SLM_001_002_20140809_1041_TIR.tif"
ndvi_file = inputdir / "SLM_001_002_20140809_1041_NDVI.tif"

#lst_file = inputdir / 'lst_surface_CC_10cm.tif'
#ndvi_file = inputdir / 'ndvi_surface_CC_10cm.tif'

# Open and read the LST file
fid = gdal.Open(str(lst_file), gdal.GA_ReadOnly)
lst = fid.GetRasterBand(1).ReadAsArray() 
# Convert Censius to Kelvin
lst = lst + 273.15
lst[lst <= 0] = None

# Get the LST GDAL geotransform and projection
gt = fid.GetGeoTransform()
proj = fid.GetProjection()
# Open and read the NDVI file
fid = gdal.Open(ndvi_file, gdal.GA_ReadOnly)
ndvi = fid.GetRasterBand(1).ReadAsArray()
ndvi[ndvi <= 0] = None

del fid

### Browse the input data
We can now visualize both images, pay special attention to the least dense rows where it is difficult to find pure vegetation pixels at 0.6m spatial resoocution, as for these areas is where the method can be specially usefull.

In [ ]:
from bokeh.plotting import *
from bokeh import palettes as pal
from bokeh.models.mappers import LinearColorMapper
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

rows, cols = int(0.3 * lst.shape[0]), int(0.3 * lst.shape[1])
# Setup the figure
map_lst = LinearColorMapper(palette=pal.RdYlBu11, high=320, low=300)
map_ndvi = LinearColorMapper(palette=pal.YlGn9, high=0.3, low=0.7)
s1 = figure(title="LST", width=cols, height=rows, x_range=[0, cols], y_range=[0, rows])
s1.axis.visible = False
s1.image(image=[np.flipud(lst)], x=[0], y=[0], dw=cols, dh=rows, color_mapper=map_lst)
s2= figure(title="NDVI", width=cols, height=rows, x_range=s1.x_range, y_range=s1.y_range)
s2.axis.visible = False
s2.image(image=[np.flipud(ndvi)], x=[0], y=[0], dw=[cols], dh=[rows], color_mapper=map_ndvi)

p = gridplot([[s1, s2]], toolbar_location='above')
show(p)

## [Nieto et al. 2022](https://doi.org/10.1007/s00271-018-0585-9) contextual method
We will obtain $T_C$ and $T_S$ by searching for pure vegetation and soil pixels in a contextual spatial domain ([](#fig-tc-ts)). That is, in a coarse resolution spatial grid, consistent with the TSEB physical assumptions, we will assign for each of its cells the canopy and soil temperatures corresponding to the average temperature for the original LST resoltuion pixels that are considered, respectively, bare soil/ and pure vegetation. 

:::{note}
In this case we assume that TSEB will be run at 3.6 m spatial resolution, as it is spatial domain large enough to include the canopy/interrow system (3.35 m width) in order to be consistent with the radiative transfer modelling in the rows, being as well a multiple of the raw LST spatial resolution of 0.6m. However, some of you could argue that even 3.6m could be a too fine spatial resolution to comply with the assumptions made during the calculation of turbulent exchange between the surface and atmosphere
:::

Pure vegetation NDVI ($NDVI_{veg}$) was set as the mean value of pixels classified as pure vegetation using a support vector machine binary supervised classification of the 0.15 m multispectral imagery ($NDVI_{veg}=0.6$). Simlarly, bare soil NDVI was set as ($NDVI_{soil}=0.3$)

:::{warning}
Note that these pure vegetation and soil NDVI values might change due to different spectral cameras and different crop/soil types.
:::

### Set our parameters in Python

In [ ]:
# Set the output spatial domain resolution at 3.6m
out_res = 3.6
# Set the NDVI value for bare soil
ndvi_soil = 0.3
# Set the NDVI value for pure vegetation
ndvi_veg = 0.6

However, it may be the case that no pure pixels at the native resolution (0.6 m in our case) are found in the coarse resolution spatial domain (3.6 m in our case), either due to very dense vegetation (e.g., lack of bare soil/substrate pixels) or very sparse vegetation (e.g., lack of pure vegetation pixels). If that is the case, and assuming that there is a linear relationship between NDVI and LST, we extrapolate to the pure vegetation or soil NDVI value the linear fit of the NDVI-LST pairs within the coarse resolution spatial domain mixed-pixel to estimate the extrapolated value for the pure vegetation (or soil) within the the aggregated pixel resolution ([](#fig-tc-ts)).

:::{figure} ./input/figures/component_temperatures.png
:alt: Contextual method for TC and TS separation
:name: fig-tc-ts
Example of contextual NDVI- LST scatterplot used for finding $T_C$ and $T_S$. Each point corresponds to a pixel NDVI-LST pair within a larger scale spatial domain. From [](https://doi.org/10.1007/s00271-018-0585-9)
:::

That is, in a 3.6m grid, we assign for each of these cells the canopy and soil temperatures corresponding to the average temperature for the 0.6 m pixels that are considered, respectively, bare soil/cover crop stubble and pure vegetation. The selection criterion for detecting pure soil NDVI is based on the empirical relationship between NDVI and in situ LAI, with NDVI is the extrapolation of that curve for . On the other hand, pure vegetation NDVI (NDVI) is the mean value of pixels classified as pure vegetation using a support vector machine binary supervised classification of the 0.15 m multispectral imagery. However, it may be the case that no pure pixels at the native resolution (0.6 m) are found in a 3.6 m spatial domain, either due to very dense vegetation (e.g., lack of bare soil/substrate pixels) or very sparse vegetation (e.g., lack of pure vegetation pixels). If that is the case, and assuming that there is a linear relationship between NDVI and , we extrapolate to the pure vegetation or soil NDVI value the linear fit of the NDVI- pairs within the 3.6 m mixed-pixel to estimate the extrapolated value for the pure vegetation (or soil) within the 3.6 m aggregated pixel resolution.



In [ ]:
# We calculate the rescaling factor based on the ratio between LST resolution (0.6) and TSEB spatial domain resolution (3.6m)
f_res = (int(np.round(np.abs(out_res / gt[5]))),
         int(np.round(np.abs(out_res / gt[1]))))
print(f"We will look for pure vegetation and bare soil pixel in a "
      f"{f_res[0]}x{f_res[0]} pixels spatial window")

Canopy (soil) temperatures are retrieved first by averaging the values above (below) a pure vegetation (soil) NDVI threshold, which corresponds to the greyed areas in the plot. If no pure pixels are found in those areas, the canopy (soil) temperature is found by extrapolating the linear fit between all NDVI- pairs in the domain to the pure vegetation (soil) NDVI threshold. 

Look at the code below that besides of the `t_leaf` and `t_leaf` outputs we also create a quality assessment output that consists on the Pearson correlation (`cor`) between the LST-NDVI pairs at 3.6 m that indicates how robust the extrapolation to pure vegetation/bare soil would be.

### Run the code to extract the $T_C$ and $T_S$ images
Depending on the size of the image this process can take a while, as it needs to loop along all the cells at which TSEB will be run. 

In [ ]:
from scipy import stats as st

# Get the lst size
nrows, ncols = lst.shape

# Initialize the outputs
out_shape = int(np.ceil(nrows / f_res[0])), int(np.ceil(ncols / f_res[1]))
t_leaf = np.full(out_shape, np.nan)
t_soil = np.full(out_shape, np.nan)
cor = np.full(out_shape, np.nan)

# Loop along all tiles
row_ini = 0
i = 0
print(f"Processing {out_shape[0] * out_shape[1]} cells, please wait...")
while row_ini < nrows - 1:
    row_end = int(np.minimum(row_ini + f_res[0], nrows))
    col_ini = 0
    j = 0
    while col_ini < ncols - 1:
        col_end = int(np.minimum(col_ini + f_res[1], ncols))
        # Get the LST values within the spatial domain
        lst_subset = lst[row_ini:row_end, col_ini:col_end]
        if ~np.any(np.isfinite(lst_subset)):
            # Jump to the next tile
            j += 1
            col_ini = int(col_end)
            continue
        
        # Get the NDVI values within the spatial domain
        vnir_subset = ndvi[row_ini:row_end, col_ini:col_end]

        # Find, if any all pure soil and vegetation pixels based on the NDVI thresholds
        soils = vnir_subset <= ndvi_soil
        vegs = vnir_subset >= ndvi_veg
        
        # Buld the local linear regression between NDVI and LST
        reg = st.linregress(np.ravel(vnir_subset), np.ravel(lst_subset))
        # Add the resulting correlation to the quality assessment band
        cor[i, j] = reg.rvalue

        
        if np.any(soils): 
            # Bare soil pixels found, computing their mean as T_S
            t_soil[i, j] = np.nanmean(lst_subset[soils])
        else:
            # No Bare soil pixels found, extrapolating using the linear regression
            t_soil[i, j] = reg.intercept + reg.slope * ndvi_soil

        if np.any(vegs):
            # Pure vegetation pixels found, computing their mean as T_C
            t_leaf[i, j] = np.nanmean(lst_subset[vegs])
        else:
            # No pure vegetation pixels found, extrapolating using the linear regression
            t_leaf[i, j] = reg.intercept + reg.slope * ndvi_veg

        # Jump to the next column tile
        j += 1
        col_ini = int(col_end)

    # Jump to the next rowntile
    i += 1
    row_ini = int(row_end)

print("Done!")

:::{seealso}
:class:dropdown
You can also check the [GitHub source code](https://github.com/hectornieto/airborne_tools/blob/5db17192e638c2745dea5d918b9dcaffd05a14cf/airborne_tools/temperature.py#L6)
:::

### We can now visualize the results:

In [ ]:
# Setup the figure
map_cor = LinearColorMapper(palette=pal.YlGn9, high=0, low=1)
s1 = figure(title="T leaf", width=cols, height=rows, x_range=[0, cols], y_range=[0, rows])
s1.axis.visible = False
s1.image(image=[np.flipud(t_leaf)], x=[0], y=[0], dw=cols, dh=rows, color_mapper=map_lst)
s2 = figure(title="T soil", width=cols, height=rows, x_range=s1.x_range, y_range=s1.y_range)
s2.axis.visible = False
s2.image(image=[np.flipud(t_soil)], x=[0], y=[0], dw=[cols], dh=[rows], color_mapper=map_lst)
s3 = figure(title="Pearson r", width=cols, height=rows, x_range=s1.x_range, y_range=s1.y_range)
s3.axis.visible = False
s3.image(image=[np.flipud(cor)], x=[0], y=[0], dw=[cols], dh=[rows], color_mapper=map_cor)
p = gridplot([[s1, s2, s3]], toolbar_location='above')
show(p)

## Save the output images
To conclude we will save our images to a 3-band GeoTIFF file, with bands:
1. Leaf Temperature (K)
2. Soil Temperature (K)
3. Local correlation between lst and vnir inputs

In [ ]:
# Set the output 
outdir = workdir / "input" / "UAV"
out_file = outdir / "SLM_001_002_20140809_1041_TC-TS.tif"

# The output GDAL geotransform shares the same UL coordinates but with 3.6m resolution 
gt_out = [gt[0], out_res, 0, gt[3], 0, -out_res]

driver = gdal.GetDriverByName("GTiff")
fid = driver.Create(str(out_file), t_leaf.shape[1], t_leaf.shape[0], 3,
                    gdal.GDT_Float32)
fid.SetProjection(proj)
fid.SetGeoTransform(gt_out)
for i, array in enumerate([t_leaf, t_soil, cor]):
    fid.GetRasterBand(i + 1).WriteArray(array)

del fid

print(f"Component temperatures save at {out_file}")

# Conclusions

* We showed a simple contextual method to extract the component soil and canopy temperatures required by TSEB models
* However this contextual method, since it is based on the LST-NDVI pairs, requires a correct corregistration between the shortwave and thermal datasets.
* In the next exercise we will see a tool that can help us to ensure the corregistration between these two products
* Finally, we will see in the final exercise how to use these data in TSEB.
* [...]

:::{note}
Please feel free to comment any thoughts. This is work in progress!!!
:::